# 특징 데이터로 유방암 진단하기

## 데이터 준비 및 탐색

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
# # 사이킷런에서 제공하는 데이터셋sklearn.datasets중에서 유방암 진단 데이터셋을 사용하기 위해 
# load_breast_cancer를 임포트

b_cancer = load_breast_cancer()
# 데이터셋을 로드하여 객체b_cancer를 생성 
# 함수 안에 데이터가 있기에, 그냥 함수만 입력하면 데이터도 다 준비됨.

In [ ]:
print(b_cancer.DESCR) # 데이터셋에 대한 설명 확인

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

:Number of Instances: 569

:Number of Attributes: 30 numeric, predictive attributes and the class

:Attribute Information:
    - radius (mean of distances from center to points on the perimeter)
    - texture (standard deviation of gray-scale values)
    - perimeter
    - area
    - smoothness (local variation in radius lengths)
    - compactness (perimeter^2 / area - 1.0)
    - concavity (severity of concave portions of the contour)
    - concave points (number of concave portions of the contour)
    - symmetry
    - fractal dimension ("coastline approximation" - 1)

    The mean, standard error, and "worst" or largest (mean of the three
    worst/largest values) of these features were computed for each image,
    resulting in 30 features.  For instance, field 0 is Mean Radius, field
    10 is Radius SE, field 20 is Worst Radius.

    - 

In [3]:
b_cancer_df = pd.DataFrame(b_cancer.data, columns = b_cancer.feature_names)
# 데이터셋 객체의 data 배열(b_cancer.data), 즉 독립 변수 X가 되는 피처를 DataFrame 자료형으로 변환하여 b_cancer_df를 생성

b_cancer_df['diagnosis']= b_cancer.target
# 유방암 유무 class로 사용할 diagnosis 컬럼을 b_cancer_df에 추가하고 
# 데이터셋 객체의 target 컬럼(b_cancer.target)을 저장
# sklearn.datasets 의 정답 데이터는 target에 저장되어 있음

b_cancer_df.head()
# b_cancer_df의 데이터 샘플 5개를 출력하여 확인

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [ ]:
print('유방암 진단 데이터셋 크기: ', b_cancer_df.shape)
# 데이터셋의 크기와 독립 변수 X가 되는 피처에 대한 정보를 확인
# 행의 개수가 569이므로 데이터 샘플이 569개, 열의 개수가 31이므로 변수가 31개 있음

유방암 진단 데이터셋 크기:  (569, 31)


In [ ]:
b_cancer_df.info() 
#b_cancer_df에 대한 정보를 확인 
# 30개의 피처(독립 변수 X)  이름과 1개의 종속 변수 이름을 확인 가능
# diagnosis는 악성이면 1, 양성이면 0의 값이므로 유방암 여부에 대한 이진 분류의 class로 사용할 종속 변수가 됨

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

### [로지스틱 회귀 분석에 피처로 사용할 데이터를 평균이 0, 분산이 1이 되는 정규 분포 형태로 맞춤]

In [ ]:
from sklearn.preprocessing import StandardScaler 
scaler = StandardScaler()
# 사이킷런의 전처리 패키지에 있는 정규 분포 스케일러를 임포트하고 사용할 객체scaler를 생성
b_cancer_scaled = scaler.fit_transform(b_cancer.data)
# 피처로 사용할 데이터b_cancer.data에 대해 정규 분포 스케일링을 수행하여 b_cancer_scaled에 저장

print(b_cancer.data[0]) #기존 scaling 전 데이터

[1.799e+01 1.038e+01 1.228e+02 1.001e+03 1.184e-01 2.776e-01 3.001e-01
 1.471e-01 2.419e-01 7.871e-02 1.095e+00 9.053e-01 8.589e+00 1.534e+02
 6.399e-03 4.904e-02 5.373e-02 1.587e-02 3.003e-02 6.193e-03 2.538e+01
 1.733e+01 1.846e+02 2.019e+03 1.622e-01 6.656e-01 7.119e-01 2.654e-01
 4.601e-01 1.189e-01]


In [ ]:
print(b_cancer_scaled[0]) # 정규 분포 스케일링 후에 값이 조정된 것을 확인

[ 1.09706398 -2.07333501  1.26993369  0.9843749   1.56846633  3.28351467
  2.65287398  2.53247522  2.21751501  2.25574689  2.48973393 -0.56526506
  2.83303087  2.48757756 -0.21400165  1.31686157  0.72402616  0.66081994
  1.14875667  0.90708308  1.88668963 -1.35929347  2.30360062  2.00123749
  1.30768627  2.61666502  2.10952635  2.29607613  2.75062224  1.93701461]


**정규 분포 형태로 맞춘 이유**
- 기존데이터는 어떤건 cm, 어떤건mm, 어떤건 좌표로 들어감
    - -> 열마다 데이터셋의 단위가 좀 다르다. 숫자의 단위와 크기가 가지각색
- 이렇게 잘 모르겠고, 일단 정규분포의 느낌을 가지는 그래프이면 fit_transform을 해준다.
- 자연데이터는 기본적으로 정규분포를 따르기에 fit_transform해주었다.
- 정규분포는 중심이 꼭 0이어야 하고, 이걸 해줘야 오작동 방지됨. 이걸 fir_transform이 알아서 맞춰줌.
- 데이터 전체를 정규분포화시킴. 그럼 사람이 보기 좀 더 편하게 결과가 나옴

## 로지스틱 회귀를 이용하여 분석모델 구축

In [8]:
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split

In [10]:
### X, Y 설정하기 
Y = b_cancer_df['diagnosis'] 
X = b_cancer_scaled
# diagnosis를 Y, 정규 분포로 스케일링한 b_cancer_scaled를 X로 설정, 테스트용 실제 암환자 결과 출력
# 간단히 말해 x값이 데이터고, Y값은 예측 결과임.(암이냐 아니냐)

### 훈련용 데이터와 평가용 데이터 분할하기 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)
# 전체 데이터 샘플 569개를 학습 데이터:평가 데이터=7:3으로 분할(test_size=0.3)함   
# random state = 0 : 데이터들이 한 쪽으로 몰려있을 수 있기에 random하게 섞음. 
# 그 후, 데이터 분할이 항상 동일하게 되도록 설정하는 시드 값을 설정.
# 이를 통해 여러 번 실행해도 같은 결과를 얻을 수 있음
print(list(Y_test))

[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1]


In [11]:
#로지스틱 회귀 분석: (1) 모델 생성 
lr_b_cancer = LogisticRegression()
# 로지스틱 회귀 분석 모델 객체 lr_b_cancer를 생성

In [12]:
#로지스틱 회귀 분석: (2) 모델 훈련
lr_b_cancer.fit(X_train, Y_train)
# 학습 데이터X_train, Y_train로 모델 학습을 수행fit( )함

LogisticRegression()

In [13]:
#로지스틱 회귀 분석: (3) 평가 데이터에 대한 예측 수행 -> 예측 결과 Y_predict 구하기
Y_predict = lr_b_cancer.predict(X_test)
# 학습이 끝난 모델에 대해 평가 데이터 X_test를 가지고 예측을 수행predict( )하여 예측값 Y_predict를 구함

## 그러니까 정리를 하면
# Xtest 데이터 중 하나 넣어줌.  > 결과가 나옴(Ypredict) >정답(Ytest 값)과 비교

## 오차행렬로 생성한 모델의 성능 확인하기

In [14]:
from sklearn.metrics import confusion_matrix, accuracy_score 
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
#오차 행렬
confusion_matrix(Y_test, Y_predict)
# 평가를 위해 7:3으로 분할한 171개의 test 데이터에 대해 이진 분류의 성능 평가 기본이 되는 오차 행렬을 구함

array([[ 60,   3],
       [  1, 107]], dtype=int64)

**오차행렬 결과**
- 실행 결과를 보면 TN이 60개, FP가 3개, FN이 1개, TP가 107개인 오차 행렬이 구해짐
- TN과 TP가 높은숫자가 나왔기에 좋은결과

In [16]:
#정확도(accuracy), 정밀도(precision), 재현율(recall), ROC_AUC
acccuracy = accuracy_score(Y_test, Y_predict) 
precision = precision_score(Y_test, Y_predict) 
recall = recall_score(Y_test, Y_predict) 
f1 = f1_score(Y_test, Y_predict) 
roc_auc = roc_auc_score(Y_test, Y_predict)
print('정확도: {0:.3f}, 정밀도: {1:.3f}, 재현율: {2:.3f}, F1: {3:.3f}'.format(acccuracy,precision,recall,f1))
# 성능 평가 지표인 정확도, 정밀도, 재현율, F1 스코어, ROC-AUC 스코어(면적의 크기)를 구함

정확도: 0.977, 정밀도: 0.973, 재현율: 0.991, F1: 0.982


In [17]:
print('ROC_AUC: {0:.3f}'.format(roc_auc))

ROC_AUC: 0.972
